In [199]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import re
from selenium.webdriver.common.by import By
from datetime import datetime 

In [10]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chromeDriver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

C:\Users\sum96\AppData\Local\Temp\ipykernel_10484\1343117472.py:5: DeprecationWarning: use options instead of chrome_options
  chromeDriver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)


In [261]:
chromeDriver.get("http://www.statiz.co.kr/schedule.php")
html = chromeDriver.page_source
bsObject = BeautifulSoup(html, 'html.parser')

# 날짜정보 받아오기
now = datetime.now().strftime("%Y-%m-%d")
now = '2023-04-09'

init_url = str(bsObject.find_all('table', {'class' : 'table table-striped table-bordered'})).split(f'boxscore.php?date={now}&amp;')[-5:]

In [262]:
batting = pd.DataFrame()

# 날짜 기준으로 해당 경기 데이터 순서대로 크롤링
for i in init_url:
    stadium = str(i).split('>')[0]
    
    # 경기 url만들기 
    url = f'http://www.statiz.co.kr/boxscore.php?opt=4&date={now}&{stadium}'
    chromeDriver.get(url)
    html = chromeDriver.page_source
    bsObject = BeautifulSoup(html, 'html.parser')
    
    
    # 홈팀
    home_batting = str(bsObject.find_all('h3')[1])
    print(home_batting)
    hometeam = home_batting.split('(')[1].split(')')[0]
    
    batting_columns = ['date', 'name', 'birth', 'team']
    for i in bsObject.find_all('th')[2:22]:
        batting_columns.append(str(i).split('<th>')[1].split('</th>')[0])
    
    
    # birth 기준으로 전처리 
    bsObject.find('table', {'class' : 'table table-striped'}).find_all('tr')[1:-1]

    batting_list = []
    for i in bsObject.find('table', {'class' : 'table table-striped'}).find_all('tr')[1:-1]:
        batting_list.append(str(i).split('birth=')[1])
               
    
    player_list = []
    for i in batting_list:
        date = now
        name = i.split('>')[1].split('<')[0]
        birth = i.split('"')[0]
        team = hometeam
        P = i.split('<td>')[1].split('</td>')[0]
        TPA = i.split('<b>')[1].split('</b>')[0]
        AB = i.split('<td>')[3].split('</td>')[0]
        R = i.split('<td>')[4].split('</td>')[0]
        H = i.split('<td>')[5].split('</td>')[0]
        HR = i.split('<td>')[6].split('</td>')[0]
        RBI = i.split('<td>')[7].split('</td>')[0]
        BB = i.split('<td>')[8].split('</td>')[0]
        HBP = i.split('<td>')[9].split('</td>')[0]   
        SO = i.split('<td>')[10].split('</td>')[0]
        GO = i.split('<td>')[11].split('</td>')[0]
        FO = i.split('<td>')[12].split('</td>')[0]
        PIT = i.split('<td>')[13].split('</td>')[0]
        GDP = i.split('<td>')[14].split('</td>')[0]
        LOB = i.split('<td>')[15].split('</td>')[0]
        AVG = i.split('<td>')[16].split('</td>')[0]
        OPS = i.split('<td>')[17].split('</td>')[0]
        LI = i.split('<td>')[18].split('</td>')[0]
        WPA = i.split('<td>')[19].split('</td>')[0]
        RE24 = i.split('<td>')[20].split('</td>')[0]
        
        player_list.append([date, name,birth,team,P,TPA,AB,R,H,HR,RBI,BB,HBP,SO,GO,FO,PIT,GDP,LOB,AVG,OPS,LI,WPA,RE24])
    team_batting = pd.DataFrame(player_list,columns=batting_columns)
    
    print(team_batting)
    
    batting = pd.concat([batting,team_batting])   
    
    away_batting = str(bsObject.find_all('h3')[2])


<h3 class="box-title"><b>타격기록 (SSG)</b></h3>
          date  name       birth team  P TPA AB  R  H HR  ... GO FO PIT GDP   
0   2023-04-09   추신수  1982-07-13  SSG  9   4  4  0  0  0  ...  2  1  15   0  \
1   2023-04-09   오태곤  1991-11-18  SSG  9   0  0  0  0  0  ...  0  0   0   0   
2   2023-04-09   최지훈  1997-07-23  SSG  8   4  4  2  2  0  ...  0  2  13   0   
3   2023-04-09    최정  1987-02-28  SSG  5   4  4  0  1  0  ...  1  2  20   0   
4   2023-04-09   최주환  1988-02-28  SSG  4   4  3  0  0  0  ...  0  2  15   0   
5   2023-04-09   김성현  1987-03-09  SSG  4   0  0  0  0  0  ...  0  0   0   0   
6   2023-04-09  에레디아  1991-01-31  SSG  7   4  4  1  2  0  ...  2  0  14   0   
7   2023-04-09   한유섬  1989-08-09  SSG  0   4  4  0  0  0  ...  0  2  14   0   
8   2023-04-09   박성한  1998-03-30  SSG  6   4  4  0  0  0  ...  3  1  16   0   
9   2023-04-09   전의산  2000-11-25  SSG  3   4  2  0  0  0  ...  1  0  16   0   
10  2023-04-09   이재원  1988-02-24  SSG  2   4  4  0  0  0  ...  1  3  12   1   

   LOB

In [263]:
batting

,date,name,birth,team,P,TPA,AB,R,H,HR,...,GO,FO,PIT,GDP,LOB,AVG,OPS,LI,WPA,RE24
0,2023-04-09,추신수,1982-07-13,SSG,9,4,4,0,0,0,...,2,1,15,0,1,0.167,0.816,0.65,-0.068,-0.95
1,2023-04-09,오태곤,1991-11-18,SSG,9,0,0,0,0,0,...,0,0,0,0,0,0.308,0.708,,0.000,0.00
2,2023-04-09,최지훈,1997-07-23,SSG,8,4,4,2,2,0,...,0,2,13,0,0,0.240,0.665,0.55,0.072,0.74
3,2023-04-09,최정,1987-02-28,SSG,5,4,4,0,1,0,...,1,2,20,0,1,0.217,0.731,0.81,-0.004,-0.14
4,2023-04-09,최주환,1988-02-28,SSG,4,4,3,0,0,0,...,0,2,15,0,1,0.263,0.702,0.59,-0.020,-0.50
5,2023-04-09,김성현,1987-03-09,SSG,4,0,0,0,0,0,...,0,0,0,0,0,0.250,0.500,,0.000,0.00
6,2023-04-09,에레디아,1991-01-31,SSG,7,4,4,1,2,0,...,2,0,14,0,1,0.333,0.907,0.83,0.136,1.34
7,2023-04-09,한유섬,1989-08-09,SSG,0,4,4,0,0,0,...,0,2,14,0,4,0.167,0.485,0.90,-0.089,-1.35
8,2023-04-09,박성한,1998-03-30,SSG,6,4,4,0,0,0,...,3,1,16,0,1,0.300,1.012,0.63,0.071,0.30
9,2023-04-09,전의산,2000-11-25,SSG,3,4,2,0,0,0,...,1,0,16,0,0,0.375,1.333,0.47,0.028,0.22


In [168]:
# P : 포지션
# TPA : 타석
# AB : 타수
# R : 득점
# H : 안타
# HR : 홈런
# RBI : 타점
# BB : 볼넷
# HBP : 데드볼
# SO : 삼진
# GO : 땅볼 아웃
# FO : ?
# PIT : ?
# GDP : 병살타
# LOB : 잔루
# AVG : 타율
# OPS : 출루율+장타율
# LI : 승부 영향 중요도
# WPA : 승리 확률 기여도
# RE24 : 기대득점 차이값


batting_columns = ['name', 'birth']
for i in bsObject.find_all('th')[2:22]:
    batting_columns.append(str(i).split('<th>')[1].split('</th>')[0])
    
    

In [98]:
# birth 기준으로 전처리 
bsObject.find('table', {'class' : 'table table-striped'}).find_all('tr')[1:-1]

batting_list = []
for i in bsObject.find('table', {'class' : 'table table-striped'}).find_all('tr')[1:-1]:
    batting_list.append(str(i).split('birth=')[1])


In [166]:

player_list = []
for i in batting_list:
    name = i.split('>')[1].split('<')[0]
    birth = i.split('"')[0]
    P = i.split('<td>')[1].split('</td>')[0]
    TPA = i.split('<b>')[1].split('</b>')[0]
    AB = i.split('<td>')[3].split('</td>')[0]
    R = i.split('<td>')[4].split('</td>')[0]
    H = i.split('<td>')[5].split('</td>')[0]
    HR = i.split('<td>')[6].split('</td>')[0]
    RBI = i.split('<td>')[7].split('</td>')[0]
    BB = i.split('<td>')[8].split('</td>')[0]
    HBP = i.split('<td>')[9].split('</td>')[0]   
    SO = i.split('<td>')[10].split('</td>')[0]
    GO = i.split('<td>')[11].split('</td>')[0]
    FO = i.split('<td>')[12].split('</td>')[0]
    PIT = i.split('<td>')[13].split('</td>')[0]
    GDP = i.split('<td>')[14].split('</td>')[0]
    LOB = i.split('<td>')[15].split('</td>')[0]
    AVG = i.split('<td>')[16].split('</td>')[0]
    OPS = i.split('<td>')[17].split('</td>')[0]
    LI = i.split('<td>')[18].split('</td>')[0]
    WPA = i.split('<td>')[19].split('</td>')[0]
    RE24 = i.split('<td>')[20].split('</td>')[0]
    
    player_list.append([name,birth,P,TPA,AB,R,H,HR,RBI,BB,HBP,SO,GO,FO,PIT,GDP,LOB,AVG,OPS,LI,WPA,RE24])




In [170]:
pd.DataFrame(player_list,columns=batting_columns)

,name,birth,P,TPA,AB,R,H,HR,RBI,BB,...,GO,FO,PIT,GDP,LOB,AVG,OPS,LI,WPA,RE24
0,추신수,1982-07-13,9,4,4,0,0,0,0,0,...,2,1,15,0,1,0.167,0.816,0.65,-0.068,-0.95
1,오태곤,1991-11-18,9,0,0,0,0,0,0,0,...,0,0,0,0,0,0.308,0.708,,0.000,0.00
2,최지훈,1997-07-23,8,4,4,2,2,0,0,0,...,0,2,13,0,0,0.240,0.665,0.55,0.072,0.74
3,최정,1987-02-28,5,4,4,0,1,0,1,0,...,1,2,20,0,1,0.217,0.731,0.81,-0.004,-0.14
4,최주환,1988-02-28,4,4,3,0,0,0,0,1,...,0,2,15,0,1,0.263,0.702,0.59,-0.020,-0.50
5,김성현,1987-03-09,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0.250,0.500,,0.000,0.00
6,에레디아,1991-01-31,7,4,4,1,2,0,1,0,...,2,0,14,0,1,0.333,0.907,0.83,0.136,1.34
7,한유섬,1989-08-09,0,4,4,0,0,0,0,0,...,0,2,14,0,4,0.167,0.485,0.90,-0.089,-1.35
8,박성한,1998-03-30,6,4,4,0,0,0,0,0,...,3,1,16,0,1,0.300,1.012,0.63,0.071,0.30
9,전의산,2000-11-25,3,4,2,0,0,0,0,2,...,1,0,16,0,0,0.375,1.333,0.47,0.028,0.22
